# Working Doc, not Report Doc

## Commenced 20 Aug 2018

## Steps to do

> 1. Import the two data files (predictors and targets) and merge, with pandas.
> 2. Cut down to the predictors to be used. 
> 3. Create composite variables.
> 4. Preliminary data exploration:
>> Categorical variables: check number of categories including blanks; create filled bar charts.<br>
>> Quantitative variables: get statistics such as mean, median, sd, range, # blanks; create violin or box plots.
> 5. Decide on further data mgt., such as: "rare" categories, binned categories, delete all rows with blanks/if too many blanks.
> 6. Train/test split.
> 7. Create decision tree as a baseline reference for random forest.
> 8. Create random forest. Tune parameters, consider boosting/bagging, confusion matrix, what is performance?
> 9. Predict on test set.
> 10. What do I now know? What does everything mean?

### 1. Import data files and merge

In [1]:
# import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# import data files
predictors = pd.read_csv('/Users/RAhmed/data store/Wesleyan_Capstone/4910797b-ee55-40a7-8668-10efd5c1b960.csv')
targets = pd.read_csv('/Users/RAhmed/data store/Wesleyan_Capstone/0bf8bc6e-30d0-4c50-956a-603fc693d966.csv')

# check loaded properly
predictors.head(2)

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,...,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
0,69572,6000.0,2011-03-14,Roman,1390,Roman,34.938093,-9.856322,none,0,...,annually,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe
1,8776,0.0,2013-03-06,Grumeti,1399,GRUMETI,34.698766,-2.147466,Zahanati,0,...,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe


In [3]:
# check loaded properly
targets.head(2)

,id,status_group
0,69572,functional
1,8776,functional


In [4]:
# merge predictors and targets, and check
full_data = pd.merge(predictors, targets, on = 'id')
full_data.head(2)

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,...,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group,status_group
0,69572,6000.0,2011-03-14,Roman,1390,Roman,34.938093,-9.856322,none,0,...,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe,functional
1,8776,0.0,2013-03-06,Grumeti,1399,GRUMETI,34.698766,-2.147466,Zahanati,0,...,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,functional


### 2A. Cut down to the predictors to be used 

Will initially keep longitude and latitude, as will use these to find nearest neighbour for gps_height where that equals zero (i.e. not known/recorded)

In [5]:
# list columns to keep
keep_cols = {'id', 'date_recorded', 'gps_height', 'installer', 'longitude', 'latitude', 'basin', 'region_code', 
             'population', 'public_meeting', 'scheme_management', 'permit', 'construction_year',
             'extraction_type_group', 'management_group', 
             'payment_type', 'water_quality', 'quantity_group', 'source_type', 
             'source_class', 'waterpoint_type_group', 'status_group'}
all_cols = set(full_data.head(0))
# create columns to drop and check with a print
drop_cols = all_cols - keep_cols
print(drop_cols)

{'management', 'ward', 'quantity', 'source', 'subvillage', 'num_private', 'payment', 'extraction_type', 'extraction_type_class', 'amount_tsh', 'district_code', 'wpt_name', 'lga', 'quality_group', 'region', 'recorded_by', 'funder', 'waterpoint_type', 'scheme_name'}


In [6]:
# create data set with just wanted variables
select_data = full_data.drop(drop_cols, axis = 1)

In [7]:
select_data.head(2)

,id,date_recorded,gps_height,installer,longitude,latitude,basin,region_code,population,public_meeting,...,construction_year,extraction_type_group,management_group,payment_type,water_quality,quantity_group,source_type,source_class,waterpoint_type_group,status_group
0,69572,2011-03-14,1390,Roman,34.938093,-9.856322,Lake Nyasa,11,109,True,...,1999,gravity,user-group,annually,soft,enough,spring,groundwater,communal standpipe,functional
1,8776,2013-03-06,1399,GRUMETI,34.698766,-2.147466,Lake Victoria,20,280,NaN,...,2010,gravity,user-group,never pay,soft,insufficient,rainwater harvesting,surface,communal standpipe,functional


### 2B. Need to make most predictors into a categorical variables

In [8]:
categories = {'installer', 'basin', 'region_code', 
             'population', 'public_meeting', 'scheme_management', 'permit',
             'extraction_type_group', 'management_group', 'payment_type', 
              'water_quality', 'quantity_group', 'source_type', 
             'source_class', 'waterpoint_type_group', 'status_group'}

for item in categories:
    select_data[item] = select_data[item].astype('category')
# check types
select_data.dtypes

id                          int64
date_recorded              object
gps_height                  int64
installer                category
longitude                 float64
latitude                  float64
basin                    category
region_code              category
population               category
public_meeting           category
scheme_management        category
permit                   category
construction_year           int64
extraction_type_group    category
management_group         category
payment_type             category
water_quality            category
quantity_group           category
source_type              category
source_class             category
waterpoint_type_group    category
status_group             category
dtype: object

### 3A. Create season_recorded, that takes category from month of recording

In [9]:
# initial testing
for item in select_data['date_recorded'][3]:
    print(item, type (item))
select_data['date_recorded'][3][5] + select_data['date_recorded'][3][6]

2 <class 'str'>
0 <class 'str'>
1 <class 'str'>
3 <class 'str'>
- <class 'str'>
0 <class 'str'>
1 <class 'str'>
- <class 'str'>
2 <class 'str'>
8 <class 'str'>


'01'

In [10]:
# create seasons as per: https://www.tripsavvy.com/tanzania-weather-and-average-temperatures-4071465
short_dry = {'01', '02'}
long_dry= {'06', '07', '08', '09', '10'}
short_rainy = {'11', '12'}
long_rainy = {'03', '04', '05'}

# create function to allocate seasons
def season(month_str):
    if month_str in short_dry:
        return 'short_dry'
    elif month_str in long_dry: 
        return 'long_dry'
    elif month_str in short_rainy: 
        return 'short_rainy'
    else:
        return 'long_rainy'

# create series for season based on date (= month) recorded, and check type is category
df = select_data['date_recorded'].apply(lambda row: season(row[5] + row[6]))
df = df.astype('category')
df.dtypes

# insert into select_data
select_data.insert(2, 'season_recorded', df)

In [11]:
# check works
select_data.head(10)

,id,date_recorded,season_recorded,gps_height,installer,longitude,latitude,basin,region_code,population,...,construction_year,extraction_type_group,management_group,payment_type,water_quality,quantity_group,source_type,source_class,waterpoint_type_group,status_group
0,69572,2011-03-14,long_rainy,1390,Roman,34.938093,-9.856322,Lake Nyasa,11,109,...,1999,gravity,user-group,annually,soft,enough,spring,groundwater,communal standpipe,functional
1,8776,2013-03-06,long_rainy,1399,GRUMETI,34.698766,-2.147466,Lake Victoria,20,280,...,2010,gravity,user-group,never pay,soft,insufficient,rainwater harvesting,surface,communal standpipe,functional
2,34310,2013-02-25,short_dry,686,World vision,37.460664,-3.821329,Pangani,21,250,...,2009,gravity,user-group,per bucket,soft,enough,dam,surface,communal standpipe,functional
3,67743,2013-01-28,short_dry,263,UNICEF,38.486161,-11.155298,Ruvuma / Southern Coast,90,58,...,1986,submersible,user-group,never pay,soft,dry,borehole,groundwater,communal standpipe,non functional
4,19728,2011-07-13,long_dry,0,Artisan,31.130847,-1.825359,Lake Victoria,18,0,...,0,gravity,other,never pay,soft,seasonal,rainwater harvesting,surface,communal standpipe,functional
5,9944,2011-03-13,long_rainy,0,DWE,39.172796,-4.765587,Pangani,4,1,...,2009,submersible,user-group,per bucket,salty,enough,other,unknown,communal standpipe,functional
6,19816,2012-10-01,long_dry,0,DWSP,33.362410,-3.766365,Internal,17,0,...,0,swn 80,user-group,never pay,soft,enough,borehole,groundwater,hand pump,non functional
7,54551,2012-10-09,long_dry,0,DWE,32.620617,-4.226198,Lake Tanganyika,17,0,...,0,nira/tanira,user-group,unknown,milky,enough,shallow well,groundwater,hand pump,non functional
8,53934,2012-11-03,short_rainy,0,Water Aid,32.711100,-5.146712,Lake Tanganyika,14,0,...,0,india mark ii,user-group,never pay,salty,seasonal,borehole,groundwater,hand pump,non functional
9,46144,2011-08-03,long_dry,0,Artisan,30.626991,-1.257051,Lake Victoria,18,0,...,0,nira/tanira,user-group,never pay,soft,enough,shallow well,groundwater,hand pump,functional


### 3B. Fill in missing gps_height variables

Approach will be to find nearest 3 neighbours and take average.

In [12]:
# testing how loc and iloc work
print(select_data.loc[0])
print(select_data.loc[0].latitude) # <= this is key technique
for row in select_data:
    print(row)
print(select_data.iloc[0])

id                                    69572
date_recorded                    2011-03-14
season_recorded                  long_rainy
gps_height                             1390
installer                             Roman
longitude                           34.9381
latitude                           -9.85632
basin                            Lake Nyasa
region_code                              11
population                              109
public_meeting                         True
scheme_management                       VWC
permit                                False
construction_year                      1999
extraction_type_group               gravity
management_group                 user-group
payment_type                       annually
water_quality                          soft
quantity_group                       enough
source_type                          spring
source_class                    groundwater
waterpoint_type_group    communal standpipe
status_group                    

In [13]:
# function to find key of maximum value in a dictionary
# will be needed for the finding indices of dictionaries for nearest gps
def find_dict_max(dict_):
    for keys, values in dict_.items():
        if values == max(dict_.values()):
            return keys

In [15]:
# will work on a smaller dataframe to make easier to follow. 
# need to make a deep copy, else original df also impacted.
import copy
df = copy.deepcopy(select_data[:20])

# initially done as code, not a function
for i in range(df['gps_height'].shape[0]):
    if df['gps_height'][i] != 0: 
        pass
    else:
        neighbours = {-3:99999, -2:99999, -1:99999}
        for j in range(df['gps_height'].shape[0]):
            if j != i:
                lat_i = df['latitude'][i]
                lat_j = df['latitude'][j]
                long_i = df['longitude'][i]
                long_j = df['longitude'][j]
                euclidian_dist = ((lat_i - lat_j)**2 + (long_i - long_j)**2)**0.5
                # check if euclidian distance small enough to go into neighbours
                if euclidian_dist < max(neighbours.values()):
                    # if yes, need to remove max distance so far and return this distance
                    neighbours.pop(find_dict_max(neighbours))
                    neighbours.update({j: euclidian_dist})
                else:
                    pass
                
        # now need to get average heights using indices in the neighbours keys
        heights = []
        for k in neighbours:
            heights.append(df['gps_height'][k])
        average_height = sum(heights)/len(heights)
        df['gps_height'][i] = average_height  
        
df.head()

/Users/RAhmed/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,id,date_recorded,season_recorded,gps_height,installer,longitude,latitude,basin,region_code,population,...,construction_year,extraction_type_group,management_group,payment_type,water_quality,quantity_group,source_type,source_class,waterpoint_type_group,status_group
0,69572,2011-03-14,long_rainy,1390,Roman,34.938093,-9.856322,Lake Nyasa,11,109,...,1999,gravity,user-group,annually,soft,enough,spring,groundwater,communal standpipe,functional
1,8776,2013-03-06,long_rainy,1399,GRUMETI,34.698766,-2.147466,Lake Victoria,20,280,...,2010,gravity,user-group,never pay,soft,insufficient,rainwater harvesting,surface,communal standpipe,functional
2,34310,2013-02-25,short_dry,686,World vision,37.460664,-3.821329,Pangani,21,250,...,2009,gravity,user-group,per bucket,soft,enough,dam,surface,communal standpipe,functional
3,67743,2013-01-28,short_dry,263,UNICEF,38.486161,-11.155298,Ruvuma / Southern Coast,90,58,...,1986,submersible,user-group,never pay,soft,dry,borehole,groundwater,communal standpipe,non functional
4,19728,2011-07-13,long_dry,387,Artisan,31.130847,-1.825359,Lake Victoria,18,0,...,0,gravity,other,never pay,soft,seasonal,rainwater harvesting,surface,communal standpipe,functional


The above code works nicely. The solution is to have a function that you run several times to get rid of zeros.

The issue you then run into is that applying the function on a row in the way we did with the season_recorded doesn't work. This is because you need to iterate over every row in any one row's function. This can be solved if you **apply the function to the whole dataframe,** as then you get iterate over i, j, etc.

One complication as that (for some reason) could not change the df['gps_height'] in situ. Had to create another df for the result, and swap it out for the old one.

Two points to note: i. For some height=0, their 3 nearest neighbours are also 0, so you still get zero entries; ii. Quite a number of longitudes are also zero, so also adjusted for that.  (i. can be solved by running function, e.g., twice.  N.B. As lat and long won't be used for predictive power, no point removing those zero rows if latitude = 0.)

In [ ]:
# just testing
df.loc[0]['gps_height']

In [ ]:
# just testing
df['gps_height'][0]

Now create the function.

In [ ]:
# make as a function, so can be done twice
def gps_helper(df):
    """
    INPUT is the whole dataframe, so that you can easily traverse rows and columns
    output is a single-column df tat has the new gps_heights
    was hugely complex as (for some reason), could not directly change df's gps_height column
    so you need to drop that column and put the new one in its place
    OUTPUT is a single-column dataframe of new gps heights
    IDEA: take 3 closest neighbours where gps_height is zero, and take average height of those.
    That has to be added to the dataframe afterwards and seprately (having removed the old one with lots of zeros
    """
    # Sadly, needed to create an output df that is added later, outside this function
    list0 = [0] * df.shape[0]
    df_gps = pd.DataFrame(list0, columns=['gps_height*'])
    for i in range(df.shape[0]):
        if df.loc[i]['gps_height'] != 0:
            df_gps.loc[i]['gps_height*'] = df.loc[i]['gps_height']
            pass 
        else:
            neighbours = {-3:99999, -2:99999, -1:99999}
            for j in range(df.shape[0]):
                # Many latitudes = 0, so also adjusting for this
                if (j != i) and (df.loc[j]['latitude'] != 0):
                    lat_i = df.loc[i]['latitude']
                    lat_j = df.loc[j]['latitude']
                    long_i = df.loc[i]['longitude']
                    long_j = df.loc[j]['longitude']
                    euclidian_dist = ((lat_i - lat_j)**2 + (long_i - long_j)**2)**0.5
                    # check if euclidian distance small enough to go into neighbours
                    if euclidian_dist < max(neighbours.values()):
                        # if yes, need to remove max distance so far and return this distance
                        neighbours.pop(find_max_key(neighbours))
                        neighbours.update({j: euclidian_dist})
                    else:
                        pass

            # now need to get average heights using indices in the neighbours keys
            heights = []
            for k in neighbours:
                heights.append(df.loc[k]['gps_height'])
                
            average_height = sum(heights)/len(heights)
            z = average_height
            df_gps.loc[i]['gps_height*'] = z

        
    zero_count = 0
    for m in range(df.shape[0]):
        if df_gps.loc[m]['gps_height*'] == 0:
            zero_count += 1
    
    print("number of zero heights is: {}".format(zero_count))
    return df_gps
        

Create a small copy of select_data on which to test and check.

In [ ]:
micro_data = copy.deepcopy(select_data[:30])
temp = gps_helper(micro_data)
temp
# # drop old gps_height column and add new gps_height column (here temp)
micro_data = micro_data.drop('gps_height', axis=1)
# # Append a column to df
micro_data.insert(3, 'gps_height', temp)
micro_data.head()

**To Do:** Run gps_helper on the complete data set that you want to use. (Potentially run it twice to get rid of remaining zero entries?) **N.B.** An issue is that if you remove rows afterwards as part of your data management, you won't be able to trace the heights back.

In [ ]:
# check gps_height is correct type. 'int64' is correct.
micro_data['gps_height'].dtypes

### 3C. Fill-in missing construction_year by matching with similar observations

We want to fill in missing construction_year so that we can create an wpt_age variable. We can do this be taking a nearest neighbour approach, but simpler than Euclidian distance as only categorical variables.

There are only a small number of chosen variables. So on your iterations through the df, if construction_year is non-zero and row matches on all chosen variables, its year is added to a list.  You would take an average of all the construction_years in the list and put that into the output dataframe. Rest of function and construction would be similar to gps_helper, above. 

Chosen categorical variables would be installer, scheme_management, extraction_type_group, and whether functional.  NOTE: the latter biases the "target" into the "predictors"!!! (Justification is that similar types would be similarly functional or not.)

You could also do for lat/long. But I will leave, as well as adding complication, it is not clear that waterpumps would be installed temporally in geographical clusters.

In [ ]:
# make as a function, so can be done twice
def construction_year_helper(df):
    """
    BASED ON gps_helper function
    INPUT is the whole dataframe, so that you can easily traverse rows and columns
    OUTPUT is a single-column df that has the new construction_year
    IDEA: if construction_year is zero, iterate df to find matches for all chosen comparison variables.
    If all match, add the iterative row's year to a list and take average of list at end. Put that into 
    output df's entry.
    Is somewhat complex as cannot directly change df's construction_year values
    """
    # Sadly, need to create an output df that is added later, outside this function
    list0 = [0] * df.shape[0]
    df_year = pd.DataFrame(list0, columns=['construction_year*'])
    for i in range(df.shape[0]):
        if df.loc[i]['construction_year'] != 0:
            df_year.loc[i]['construction_year*'] = df.loc[i]['construction_year']
            pass 
        else:
            # initialise list that you will put matching years into
            matching_years = []
            for j in range(df.shape[0]):
                # Many construction_year = 0, so not using these
                if (j != i) and (df.loc[j]['construction_year'] != 0):
                    # using a counter for readability
                    counter = 0
                    if df.loc[j]['installer'] == df.loc[j]['installer']:
                        counter += 1
                    if df.loc[j]['scheme_management'] == df.loc[j]['scheme_management']:
                        counter += 1   
                    if df.loc[j]['extraction_type_group'] == df.loc[j]['extraction_type_group']:
                        counter += 1 
                    if df.loc[j]['status_group'] == df.loc[j]['status_group']:
                        counter += 1     
                    # check if all items match
                    if counter == 4:
                        # if yes, add to matching_years
                        matching_years.append(df.loc[j]['construction_year'])
                    else:
                        pass

            # now need to get average year of the matches   
            average_year = sum(matching_years)/len(matching_years)
            z = round(average_year)
            df_year.loc[i]['construction_year*'] = z

        
    zero_count = 0
    for m in range(df.shape[0]):
        if df_year.loc[m]['construction_year*'] == 0:
            zero_count += 1
    
    print("number of zero years is: {}".format(zero_count))
    return df_year

Create a small copy of select_data on which to test and check.

In [ ]:
mini_data = copy.deepcopy(select_data[:30])
temp = construction_year_helper(mini_data)
temp
# # drop old construction_year column and add new construction_year column (here temp)
mini_data = mini_data.drop('construction_year', axis=1)
# # Append a column to df
mini_data.insert(13, 'construction_year', temp)
mini_data.head()

**To Do:** Run construction_year_helper on the complete data set that you want to use. No need to run twice as seems to have no zero entries. (Shouldn't do by its construction.) 

### 3D. wpt_age variable

In [ ]:
print(type(select_data['date_recorded'][1]))
print(type(select_data['construction_year'][1]))

A significant issue was that a lot of missing contruction_year. That is now solved by the construction_year_helper, above.

Now need to change year recorded into a numerical value from a string.

In [ ]:
# just testing
x = '2013'
y = int(x)
print(y, type(y))
z = float(x)
print(z, type(z))

In [ ]:
# create helper function and test
def year_convert(date_recorded):
    """
    takes custom string (e.g. '2013-04-17') and converts to a float
    is not super accurate as assumes all months have 31 days
    
    """
    year = float(date_recorded[:4])
    month = float(date_recorded[5:7])
    day = float(date_recorded[8:])
    
    return year + ((month - 1) / 12) + (day / (31*12)) 

year_convert('2013-02-28')

Create a separate new df called wpt_age, and then add to global df.

Test on a smaller data set.

In [ ]:
df_wpt_age = mini_data['date_recorded'].apply(year_convert) - mini_data['construction_year']
df_wpt_age.head()

Now add to global dataset

In [ ]:
mini_data.insert(14, 'wpt_age', df_wpt_age)
mini_data.head()

**To Do:** Run year_convert on the complete data set that you want to use. Add the single-column df to the global dataset.

### 3E. Applying improvements to global data frame

Will use select_data as basis, making a copy.

In [ ]:
stage3_data = copy.deepcopy(select_data)

season_recorded was a simple fix, so that has already been globally implemented. Next is to fix the missing gps_height values.

In [ ]:
gps_temp = gps_helper(stage3_data)
# # drop old gps_height column and add new gps_height column (here temp)
stage3_data = stage3_data.drop('gps_height', axis=1)
# # Append a column to df
stage3_data.insert(3, 'gps_height', gps_temp)

In [ ]:
stage3_data.head()

Is taking a very long time, could consider sorting by latitude first.  Could then just check the 3,000-4,000 either side?  (Thinking about this, I've asked it to do 59,000\**2 checks, = nearly 3.5 billion checks against other variables.

For missing construction_year, could first sort by installer?

### 4. Preliminary data exploration

Will have to fix NaNs in various columns

In [ ]:
mini_data.loc[3]

In [ ]:
print(mini_data.index[1])

In [ ]:
mini_data.index

In [ ]:
mini_data.loc[1]['gps_height']